In [3]:
import numpy as np
import random
from PIL import Image
im = Image.open('elder.jpg').convert('L')
X = np.array(im)
d, n = X.shape

In [4]:
k, alpha = int(0.15 * d) + 1, 0.3
S = X.T[random.sample(range(n), int(alpha * n))]

In [11]:
m_max = 13
XS = [X.dot(np.random.normal(loc=0.0, scale=1, size=(n, m_max))) ** 2 + X.dot(np.random.normal(loc=0.0, scale=1, size=(n, m_max))) ** 2 for _ in range(k)]
XS = [np.sqrt(i) for i in XS]

In [14]:
print('NMF via OPU:')
for m in range(1, m_max + 1):
    print('t = ', m)
    Q, QT, W = np.zeros((d, 0)), np.zeros((0, d)), np.zeros((d, 0))
    for i in range(k):
        r = XS[i][:, :m]
        rr = r - Q.dot(QT.dot(r))
        rr = rr / np.sqrt(np.sum(rr ** 2, axis = 0))
        j = np.argmax(np.sum(S.dot(rr) ** 2, axis = 0))
        r, rr = r[:, j], rr[:, j]
        Q, QT, W = np.c_[Q, rr], np.r_[QT, rr.reshape(1, -1)], np.c_[W, r / np.sqrt(np.sum(r ** 2))]
    H, XX = np.zeros((k, n)), X.copy()
    for i in range(k - 1, -1, -1):
        q, w = QT[i], W[:, i]
        h = q.dot(XX) / np.sum(w * q)
        XX = XX - w.reshape(-1, 1).dot(h.reshape(1, -1))
        H[i] = h
    print('Squared Error:', np.sum((XX) ** 2))

NMF via OPU:
t =  1
Squared Error: 2980527.318902525
t =  2
Squared Error: 2120073.5904159187
t =  3
Squared Error: 2261749.3266899674
t =  4
Squared Error: 2230119.1610115375
t =  5
Squared Error: 2093730.812196142
t =  6
Squared Error: 2217346.438053477
t =  7
Squared Error: 1943104.243740218
t =  8
Squared Error: 2032956.400258094
t =  9
Squared Error: 1912040.6269805778
t =  10
Squared Error: 1714831.0590467146
t =  11
Squared Error: 1795950.362027059
t =  12
Squared Error: 1778998.3020501265
t =  13
Squared Error: 1554115.7837754241


In [16]:
def multiplication_update(V, r, k, e):
    m, n = V.shape
    W, H = np.random.random((m, r)), np.random.random((r, n))
    num_iter = 0
    for x in range (k):
        V_pre = W.dot(H)
        E = V - V_pre
        err = np.sum(E ** 2)
        if num_iter % 50 == 0:
            print('# of iter: ',x, ', error:', err)
        if err < e:
            break
        num_iter += 1
        a = W.T.dot(V)
        b = W.T.dot(W.dot(H))
        H = H * a / b
        c = V.dot(H.T)
        d = W.dot(H.dot(H.T))
        W = W * c / d
    return W,H
W, H = multiplication_update(X, k, 1000, np.sum((XX) ** 2))

# of iter:  0 , error: 1392905406.1700153
# of iter:  50 , error: 18184394.694487754
# of iter:  100 , error: 7256699.162163172
# of iter:  150 , error: 4789084.845749097
# of iter:  200 , error: 3579386.069888514
# of iter:  250 , error: 2843786.712617522
# of iter:  300 , error: 2363316.573945705
# of iter:  350 , error: 2032025.40318692
# of iter:  400 , error: 1793670.603745045
# of iter:  450 , error: 1616800.3836268855
